In [1]:
import pandas as pd
import os
mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-5.3.0-posix-seh-rt_v4-rev0\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

import warnings
warnings.filterwarnings('ignore')

Read the Data from 1 Simulation 

In [2]:
data = pd.read_csv('Test_RF.csv')
data['Label'] = data.Last.shift(-1)
data = data.dropna(axis=0)
data.head()

,Player_Last,Last,pred,Player_lag_1,Player_lag_2,Player_lag_3,Player_lag_4,Player_lag_5,Player_lag_6,Player_lag_7,...,Bot_lag_91,Bot_lag_92,Bot_lag_93,Bot_lag_94,Bot_lag_95,Bot_lag_96,Bot_lag_97,Bot_lag_98,Bot_lag_99,Label
99,7.0,8.0,4.0,3.0,2.0,3.0,5.0,1.0,7.0,4.0,...,2.0,8.0,8.0,1.0,8.0,1.0,2.0,1.0,7.0,1.0
100,1.0,1.0,5.0,7.0,3.0,2.0,3.0,5.0,1.0,7.0,...,7.0,2.0,8.0,8.0,1.0,8.0,1.0,2.0,1.0,1.0
101,9.0,1.0,7.0,1.0,7.0,3.0,2.0,3.0,5.0,1.0,...,7.0,7.0,2.0,8.0,8.0,1.0,8.0,1.0,2.0,3.0
102,6.0,3.0,3.0,9.0,1.0,7.0,3.0,2.0,3.0,5.0,...,2.0,7.0,7.0,2.0,8.0,8.0,1.0,8.0,1.0,1.0
103,9.0,1.0,7.0,6.0,9.0,1.0,7.0,3.0,2.0,3.0,...,4.0,2.0,7.0,7.0,2.0,8.0,8.0,1.0,8.0,6.0


In [3]:
data.shape

(29899, 202)

Random Forest + XGB + LR from sklearn

In [4]:
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import xgboost as xgb
import numpy as np


#x_train, x_test, y_train, y_test = train_test_split(data.drop('Label',1), data.Label, test_size = 0.3)

x = data.drop('pred',1).drop('Label',1)
x_train = x[x.index < 0.8*x.shape[0]] 
x_test = x[x.index > 0.8*x.shape[0]] 

y = data.Label
y_train = y[y.index < 0.8*y.shape[0]] 
y_test = y[y.index > 0.8*y.shape[0]] 


In [5]:
rf =RandomForestClassifier(n_estimators= 300)
gbm = xgb.XGBRegressor(max_depth=3, n_estimators=300, learning_rate=0.05).fit(x_train, y_train)
lr = LogisticRegression().fit(x_train, y_train)

rf.fit(x_train,y_train)

print "Train RF ACC:", accuracy_score(y_train, rf.predict(x_train))
print "Test RF ACC:", accuracy_score(y_test, rf.predict(x_test))

print "Train GBM ACC:", accuracy_score(y_train, np.round(gbm.predict(x_train)))
print "Test GBM ACC:", accuracy_score(y_test, np.round(gbm.predict(x_test)))

print "Train LR ACC:", accuracy_score(y_train, np.round(lr.predict(x_train)))
print "Test LR ACC:", accuracy_score(y_test, np.round(lr.predict(x_test)))

Train RF ACC: 1.0
Test RF ACC: 0.130470549523
Train GBM ACC: 0.133579614626
Test GBM ACC: 0.116321158276
Train LR ACC: 0.17564333991
Test LR ACC: 0.12372490951


Hidden Markov Model using HmmLearn

In [78]:
from hmmlearn.hmm import GaussianHMM
class HMMPlayer:
    ''' A Simple class that simulate a Random Player for the #3 Hurdle of the DARPA Competition  '''
    import numpy as np
    
        
    def __init__ (self, M, retrain=500, seq_size = 100):
        self._type = "HMM"        
        self.M = M                               ##  M-outputs
        self.record = pd.DataFrame( columns=['Player_Last','Last','pred'])
        self.turn = 0
        self.retrain = retrain

                    
    def start(self):
        
        D,P = [np.random.choice(range(1,self.M)),np.random.choice(range(1,self.M))]
        self.Prev_D = D
        return D,P
        
    def step(self, A):
        
        self.turn = self.turn +1
        self.appendData(A,self.Prev_D, self.LastPred)
        
        if self.turn < self.retrain:
            D,P = self.start()
                    
        else:
            # check if is time to train 
            if self.turn%self.retrain == 0:
                # Train
                self.Train()
                    
            D,P = self.predict()            
                 
        self.Prev_D = D
        self.LastPred = P
        return [D,P]
        
    def appendData(self, A, D,P):
                
        self.record = self.record.append(pd.DataFrame([[D,A,P]], columns=['Player_Last','Last','pred']),ignore_index=True)
        
        
    def Train(self):                
        
        print "Training.. "
        df = self.record.drop('pred',1).copy()
        df['Label'] = self.Dataset.Last.shift(-1)  
        df = df.dropna(axis=0) # Remove NA rows 
        
        
        x = df.drop('Label',1)
        y = df.Label
        
        model = GaussianHMM(n_components=10, covariance_type="full", n_iter=2).fit(x)
               
        self.clf = grid_search.fit(x,y)            
        
        #print "Train ACC:", accuracy_score(y, self.clf.predict(x))
        return True    

    
    def predict(self):
        
        # Predict the optimal sequence of internal hidden state
        hidden_states = model.predict(self.record[['Player_Last', 'Last']]) ## This will give us a n_components x n_components transmition matrix that tries to imitate the DARPA Player Q matrix
        
        darpa_state = hidden_states[-1:] # this will give me the last state of the most probable 
        p_ = model.transmat_[darpa_state,:][0]
        p_x = (1-p_)/sum(1-p_) # Inverse Probabilities to get the Least Likely        
        
        P = np.random.choice(range(1,self.N), p=p_)
        D = np.random.choice(range(1,self.N), p= p_x)

        
        return D,P  # P = More Likely, D = Less Likely 
        
    def restart(self):
        print "Restarting.. "
        self.__init__(self.N,retrain=self.retrain)
        return True
        

In [46]:
print("fitting to HMM and decoding ...")

# Make an HMM instance and execute fit
model = GaussianHMM(n_components=10, covariance_type="full", n_iter=2).fit(x_train[['Player_Last', 'Last']])



fitting to HMM and decoding ...


In [53]:
# Predict the optimal sequence of internal hidden state
        hidden_states = model.predict(self.record[['Player_Last', 'Last']]) ## This will give us a n_components x n_components transmition matrix that tries to imitate the DARPA Player Q matrix

print("done")

done


In [84]:
darpa_state = hidden_states[-1:] # this will give me the last state of the most probable 
p_ = model.transmat_[darpa_state,:][0]
p_x = (1-p_)/sum(1-p_)
P = np.random.choice(range(10), p=p_)

p_x = 1-p_
p_x[P] = 0
print p_x
p_x = p_x/sum(p_x)
print np.random.choice(range(10), p=p_x)

print p_
print P


[ 0.09649138  0.11419592  0.0995999   0.08901213  0.10448929  0.10347391
  0.08244851  0.10974098  0.07908446  0.12146352]
9


[ 0.90350862  0.88580408  0.9004001   0.91098787  0.89551071  0.89652609
  0.91755149  0.89025902  0.92091554  0.        ]
5


LSTM on Tensorflow


Example of Predicting a Sin(x)

In [6]:
%matplotlib inline
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt

from tensorflow.contrib import learn
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

from lstm import generate_data, lstm_model, prepare_data


In [19]:
LOG_DIR = './ops_logs/sin'
TIMESTEPS = 20
RNN_LAYERS = [{'num_units': 5}]
DENSE_LAYERS = 0
TRAINING_STEPS = 30000
PRINT_STEPS = TRAINING_STEPS / 10
BATCH_SIZE = 100

In [20]:
regressor = learn.Estimator(model_fn=lstm_model(TIMESTEPS, RNN_LAYERS, DENSE_LAYERS),
                            model_dir=LOG_DIR)

In [21]:
min_max_scaler = MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(data['Last'])
x_norm = pd.DataFrame(X_train_minmax)

train_x, val_x, test_x = prepare_data(x_norm , TIMESTEPS)
train_y, val_y, test_y = prepare_data(x_norm , TIMESTEPS, labels=True)

X, y = dict(train=train_x, val=val_x, test=test_x), dict(train=train_y, val=val_y, test=test_y)

/home/bts/.local/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/bts/.local/lib/python2.7/site-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [22]:
#X, y = generate_data(np.sin, np.linspace(0, 100, 10000, dtype=np.float32), TIMESTEPS, seperate=False)

In [23]:
print X['train'].shape, X['test'].shape
#print X['train']

print y['train'].shape, y['test'].shape
#print y['train']

(24198, 20, 1) (2970, 20, 1)
(24198, 1) (2970, 1)


In [24]:


# create a lstm instance and validation monitor
validation_monitor = learn.monitors.ValidationMonitor(X['val'], y['val'],
                                                     every_n_steps=PRINT_STEPS,
                                                     early_stopping_rounds=1000)
# print(X['train'])
# print(y['train'])

regressor.fit(X['train'], y['train'], 
              monitors=[validation_monitor], 
              batch_size=BATCH_SIZE,
              steps=TRAINING_STEPS)

Instructions for updating:
Use mean_squared_error.
Instructions for updating:
Use mean_squared_error.


Estimator(params=None)

In [ ]:
predicted = regressor.predict(X['test'])
rmse = np.sqrt(((predicted - y['test']) ** 2).mean(axis=0))
score = mean_squared_error(predicted, y['test'])
print ("MSE: %f" % score)

In [ ]:
plot_predicted, = plt.plot(predicted[0:500], label='predicted')
plot_test, = plt.plot(y['test'][0:500], label='test')
plt.legend(handles=[plot_predicted, plot_test])

What I need to do is format the input and output the same way